<a href="https://colab.research.google.com/github/Trading-com-Dados/desenvolvimento_interno/blob/main/JPJ/MythBusters_do_Mercado_Financeiro_timeframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

---
# **MythBusters do Mercado Financeiro: dividendos**
---

# **1) Bibliotecas utilizadas**

In [1]:
!pip install yfinance
!pip install ta
!pip install backtrader

     |████████████████████████████████| 63 kB 881 kB/s 
     |████████████████████████████████| 6.4 MB 21.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28985 sha256=63c74a56426719d79d4cc1595ab6535639fd9ba0f15e7e860b4a293e4455b8fb
  Stored in directory: /root/.cache/pip/wheels/bc/2a/c2/a56e77d07edc16a

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import pytz

import yfinance as yf
import ta
import backtrader as bt

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# %matplotlib inline

# **2. Criar o dataset**

## **2.1. Yahoo Finance - yfinance**

In [9]:
## Importar os dados de cotações do Yahoo Finance

df_1WK = yf.download('BTC-USD', period='max', interval = '1wk')
df_1D = yf.download('BTC-USD', period='max', interval = '1d')
df_5min = yf.download('BTC-USD', start='2022-04-18', end = '2022-04-24', interval = '5m')
# df_1D = yf.download('BTC-USD', start='2021-01-01', end = '2021-03-31', interval = '1d')
df_1WK

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-15,465.864014,468.174011,384.532013,398.821014,398.821014,156903400
2014-09-22,399.100006,441.557007,374.332001,377.181000,377.181000,186772600
2014-09-29,376.928009,391.378998,289.295990,320.510010,320.510010,276657896
2014-10-06,320.389008,382.726013,302.559998,378.549011,378.549011,341152804
2014-10-13,377.920990,411.697998,368.897003,389.545990,389.545990,156902070
...,...,...,...,...,...,...
2022-04-04,46445.273438,47106.140625,42021.207031,42207.671875,42207.671875,188557001876
2022-04-11,42201.039062,42424.589844,39373.058594,39716.953125,39716.953125,174652159709
2022-04-18,39721.203125,42893.582031,38696.191406,39469.292969,39469.292969,184314843516


## **2.2. Ilustrações do contexto**

In [72]:
fig = make_subplots(rows=2, cols=1)

cotacoes_BTC_yf_fig_dia = df_1D[(df_1D.index > '2022-04-20') & (df_1D.index <= '2022-04-22')]
cotacoes_BTC_yf_fig_min = df_5min[(df_5min.index > '2022-04-21') & (df_5min.index <= '2022-04-22')]

fig.add_trace(go.Candlestick(name="Bitcoin",x=cotacoes_BTC_yf_fig_dia.index, open=cotacoes_BTC_yf_fig_dia['Open'],high=cotacoes_BTC_yf_fig_dia['High'],
                             low=cotacoes_BTC_yf_fig_dia['Low'],close=cotacoes_BTC_yf_fig_dia['Close'],increasing_line_color= 'green', decreasing_line_color= 'red',showlegend=False), row=1,col=1)

fig.add_trace(go.Candlestick(name="Bitcoin",x=cotacoes_BTC_yf_fig_min.index, open=cotacoes_BTC_yf_fig_min['Open'],high=cotacoes_BTC_yf_fig_min['High'],
                             low=cotacoes_BTC_yf_fig_min['Low'],close=cotacoes_BTC_yf_fig_min['Close'],increasing_line_color= 'green', decreasing_line_color= 'red',showlegend=False), row=2,col=1)

fig.update_traces(line_width=3)

fig.update_yaxes(title_text="<b> Bitcoin USD")

fig.update_layout(xaxis_rangeslider_visible=False,xaxis2_rangeslider_visible=False,title_text='',template = 'simple_white',margin=dict(l=20, r=20, t=20, b=20), 
                  paper_bgcolor="#f7f8fa",width=500,height=800,font=dict(family="Arial",size=20,color="black"),xaxis=dict(range=['2022-04-20', '2022-04-22']),
                  xaxis2=dict(range=['2022-04-21', '2022-04-22']),yaxis=dict(range=['40000', '43000']),yaxis2=dict(range=['40000', '43000']))

fig.show()

In [110]:
cotacoes_BTC_yf_fig = df_1D[(df_1D.index > '2020-12-16') & (df_1D.index <= '2021-04-18')]

fig = go.Figure(data=[go.Candlestick(go.Candlestick(name='Bitcoin_USD', x=cotacoes_BTC_yf_fig.index, open=cotacoes_BTC_yf_fig['Open'],high=cotacoes_BTC_yf_fig['High'],
                                                    low=cotacoes_BTC_yf_fig['Low'],close=cotacoes_BTC_yf_fig['Close'],increasing_line_color= 'green', decreasing_line_color= 'red'))])

fig.update_traces(line_width=1)
fig.update_yaxes(title_text="<b> Bitcoin USD")
fig.update_layout(xaxis_rangeslider_visible=False,title_text='',template = 'simple_white',margin=dict(l=20, r=20, t=20, b=20), paper_bgcolor="#f7f8fa",width=1000,height=400,
                  font=dict(family="Arial",size=20,color="black"),
                  #xaxis=dict(range=['2021-10-18', '2022-04-21'])
                  )

fig.show()